In [ ]:
import cv2
import numpy as np
import os
import mediapipe as mp

from sklearn.model_selection import train_test_split

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow import keras

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from scipy import stats
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

from jproperties import Properties

In [ ]:
configs = Properties()
with open('../.properties', 'rb') as config_file: configs.load(config_file)
CAMERA_INDEX = int(configs.get('CAMERA_INDEX').data)

In [ ]:
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic()

In [ ]:
DATA_PATH = os.path.join('dist') 
num_videos = 20
sequence_length = 60

if not os.path.exists(DATA_PATH):
    os.mkdir(DATA_PATH)

In [ ]:
# clear dist
for file in os.listdir(DATA_PATH):
    os.remove(os.path.join(DATA_PATH, file))

# Registraiamo video

In [ ]:
cap = cv2.VideoCapture(CAMERA_INDEX)

width= int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height= int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

width= int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height= int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

for idx_video in range(num_videos):

    video_filename = os.path.join(DATA_PATH, f"{idx_video}.mp4")
    writer= cv2.VideoWriter(video_filename, cv2.VideoWriter_fourcc(*'DIVX'), 20, (width,height))

    for idx_frame in range(sequence_length):
        ret, frame = cap.read()

        if idx_frame == 0:
            cv2.putText(frame, f'Stating Video #{idx_video+1}', (120,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
            cv2.imshow('OpenCV Feed', frame)
            cv2.waitKey(3000)
        else:
            cv2.imshow('OpenCV Feed', frame)

        writer.write(frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
    writer.release()
                
cap.release()
cv2.destroyAllWindows()

## Mostriamo e basta

In [ ]:
cap = cv2.VideoCapture(CAMERA_INDEX)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Converti il frame in scala di grigi
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Esegui il rilevamento delle pose con MediaPipe
    results = holistic.process(frame_rgb)

    # Estrai le coordinate della spalla destra, del gomito destro e della mano destra
    if results.pose_landmarks:
        right_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER]
        right_elbow = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW]
        right_wrist = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST]

        # Le coordinate sono normalizzate, quindi convertile in coordinate pixel
        height, width, _ = frame.shape
        shoulder_x, shoulder_y = int(right_shoulder.x * width), int(right_shoulder.y * height)
        elbow_x, elbow_y = int(right_elbow.x * width), int(right_elbow.y * height)
        wrist_x, wrist_y = int(right_wrist.x * width), int(right_wrist.y * height)

        # Disegna i punti sul frame
        cv2.circle(frame, (shoulder_x, shoulder_y), 5, (0, 0, 255), -1)
        cv2.circle(frame, (elbow_x, elbow_y), 5, (0, 255, 0), -1)
        cv2.circle(frame, (wrist_x, wrist_y), 5, (255, 0, 0), -1)

    # Visualizza il frame con le coordinate
    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


## Modello

In [ ]:
window_len = 10
X_training = []
y_training = []
frame_buffer = []
video_paths = [os.path.join(DATA_PATH, f"{idx_video}.mp4") for idx_video in range(num_videos)]

for video_path in video_paths:
    cap = cv2.VideoCapture(video_path)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(frame_rgb)

        if results.pose_landmarks:
            #right_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER]
            #right_elbow = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW]
            right_wrist = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST]
            
            # Estrai le coordinate x, y, e z e normalizzale
            #shoulder_coords = [right_shoulder.x, right_shoulder.y, right_shoulder.z]
            #elbow_coords = [right_elbow.x, right_elbow.y, right_elbow.z]
            wrist_coords = [
                right_wrist.x, 
                right_wrist.y, 
                #right_wrist.z
                ]

            frame_buffer.append(
                # shoulder_coords
                #  + elbow_coords 
                wrist_coords
            )

            # Se il buffer contiene abbastanza frame, aggiungilo a X_training e y_training
            if len(frame_buffer) >= window_len:
                X_training.append(frame_buffer[:int(window_len/2)])
                y_training.append(frame_buffer[-1])
                frame_buffer = frame_buffer[1:]

        cv2.imshow('OpenCV Feed', frame)

    cap.release()


In [ ]:
X_training = np.array(X_training)
y_training = np.array(y_training)
X_training.shape, y_training.shape

In [ ]:
# Define the model
model = Sequential()

# Add an LSTM layer with input shape (.., ..)
model.add(LSTM(64, input_shape=(X_training.shape[1], X_training.shape[2]), activation='relu', return_sequences=True))

# Add another LSTM layer with return_sequences=False for the final prediction
model.add(LSTM(64, activation='relu', return_sequences=False))

# Add a dense output layer with 9 units (assuming you want to predict 9 values)
model.add(Dense(2))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Print the model summary
model.summary()

In [ ]:
batch_size = 32
epochs = 50

# Train the model
history = model.fit(X_training, y_training, batch_size=batch_size, epochs=epochs, validation_split=0.2)

In [ ]:
cap = cv2.VideoCapture(CAMERA_INDEX)
frame_buffer = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Converti il frame in scala di grigi
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Esegui il rilevamento delle pose con MediaPipe
    results = holistic.process(frame_rgb)

    # Estrai le coordinate della spalla destra, del gomito destro e della mano destra
    if results.pose_landmarks:
        right_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER]
        right_elbow = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW]
        right_wrist = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST]

        # Le coordinate sono normalizzate, quindi convertile in coordinate pixel
        height, width, _ = frame.shape
        # shoulder_x, shoulder_y = int(right_shoulder.x * width), int(right_shoulder.y * height)
        # elbow_x, elbow_y = int(right_elbow.x * width), int(right_elbow.y * height)
        wrist_x, wrist_y = int(
            right_wrist.x * width), int(right_wrist.y * height)

        # Disegna i punti sul frame
        # cv2.circle(frame, (shoulder_x, shoulder_y), 5, (0, 0, 255), -1)
        # cv2.circle(frame, (elbow_x, elbow_y), 5, (0, 255, 0), -1)
        cv2.circle(frame, (wrist_x, wrist_y), 5, (255, 0, 0), -1)

        # Estrai le coordinate x, y, e z e normalizzale
        # shoulder_coords = [right_shoulder.x, right_shoulder.y, right_shoulder.z]
        # elbow_coords = [right_elbow.x, right_elbow.y, right_elbow.z]
        wrist_coords = [
            right_wrist.x,
            right_wrist.y,
            # right_wrist.z
        ]
        print(wrist_coords)

        # Logica per la predizione
        frame_buffer.append(
            # shoulder_coords
            # + elbow_coords
            wrist_coords
        )

        if len(frame_buffer) >= window_len/2:
            prediction_input = np.array(frame_buffer)[np.newaxis, :, :]
            prediction = model.predict(prediction_input)
            print(prediction)

            # Draw magenta circle for the predicted wrist position
            prediction_x = int(prediction[0][0] * width)
            prediction_y = int(prediction[0][1] * height)
            cv2.circle(frame, (prediction_x, prediction_y),
                       5, (255, 0, 255), -1)
            
            #Draw an arrow between the predicted and actual wrist position
            cv2.arrowedLine(frame, (wrist_x, wrist_y), (prediction_x, prediction_y), (255, 0, 255), 2)

            frame_buffer = frame_buffer[1:]

    # Visualizza il frame con le coordinate
    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()